## test code

In [1]:
#calculate IoU of the prediction and the ground truth

import os
import numpy as np
from PIL import Image

pred_dir = "/home/fk/python_code/ltqs/weights_bj_baseline/dlink34_sat_gpsdata_png_count_direct_0_dice_bce_loss_1.0__/prediction_img"
# pred_dir = "/home/fk/python_code/ltqs/weights_bj_baseline/dlink34_sat_gpsimage_png_filtered_gaussian_ltqs_log_0_dice_bce_loss_1.0__/prediction_img"
label_dir = "/home/fk/python_code/datasets/dataset_bj_time/test/mask"

sum_IoU = 0
pred_list = os.listdir(pred_dir)
for pred_file in pred_list:
    pred_path = os.path.join(pred_dir, pred_file)
    label_file = pred_file.split(".")[0] + "_mask.png"
    label_path = os.path.join(label_dir, label_file)
    pred = np.array(Image.open(pred_path))
    label = np.array(Image.open(label_path))
    pred = pred > 0
    label = label > 0
    intersection = np.logical_and(pred, label)
    union = np.logical_or(pred, label)
    iou = np.sum(intersection) / np.sum(union)
    sum_IoU += iou
ave_IoU = sum_IoU / len(pred_list)
print(ave_IoU)

0.6010629241608954


In [9]:
import os
import numpy as np
from PIL import Image

pred_dir = "/home/fk/python_code/ltqs/weights_bj_baseline/dlink34_sat_gpsdata_png_count_direct_0_dice_bce_loss_1.0__/prediction_npy"
label_dir = "/home/fk/python_code/datasets/dataset_bj_time/test/mask"

sum_IoU = 0
pred_list = os.listdir(pred_dir)
for pred_file in pred_list:
    pred_path = os.path.join(pred_dir, pred_file)
    label_file = pred_file.split(".")[0] + "_mask.png"
    label_path = os.path.join(label_dir, label_file)
    pred = np.load(pred_path)
    label = np.array(Image.open(label_path))
    pred = pred > 0.5
    label = label > 0.5
    intersection = np.logical_and(pred, label)
    union = np.logical_or(pred, label)
    iou = np.sum(intersection) / np.sum(union)
    sum_IoU += iou
ave_IoU = sum_IoU / len(pred_list)
print(ave_IoU)

0.6010629241608952


In [11]:
import torch
import torch.nn as nn

class IoU(nn.Module):
    def __init__(self, threshold=0.5):
        super(IoU, self).__init__()
        self.threshold = threshold

    def forward(self, target, input):
        eps = 1e-10
        input_ = (input > self.threshold).data.float() # If the value of the pixel is greater than the threshold, then the pixel is 1, otherwise it is 0.
        target_ = (target > self.threshold).data.float()

        intersection = torch.clamp(input_ * target_, 0, 1)
        union = torch.clamp(input_ + target_, 0, 1)

        if torch.mean(intersection).lt(eps): #less than函数
            return torch.Tensor([0., 0., 0., 0., 0.])
        else:
            acc = torch.mean((input_ == target_).data.float()) # accuracy
            iou = torch.mean(intersection) / torch.mean(union)
            recall = torch.mean(intersection) / torch.mean(target_)
            precision = torch.mean(intersection) / torch.mean(input_)
            f1 = 2 * recall * precision / (recall + precision) # F1-score

            return torch.Tensor([acc, recall, precision, iou, f1])

iou = IoU()
pred_dir = "/home/fk/python_code/ltqs/weights_bj_baseline/dlink34_sat_gpsdata_png_count_direct_0_dice_bce_loss_1.0__/prediction_npy"
label_dir = "/home/fk/python_code/datasets/dataset_bj_time/test/mask"
pred_list = os.listdir(pred_dir)
sum_IoU = 0
sum_f1 = 0
for pred_file in pred_list:
    pred_path = os.path.join(pred_dir, pred_file)
    label_file = pred_file.split(".")[0] + "_mask.png"
    label_path = os.path.join(label_dir, label_file)
    pred = torch.Tensor(np.load(pred_path))
    label = torch.Tensor(np.array(Image.open(label_path)))

    iou_value = iou(label, pred)
    sum_IoU += iou_value[3]
    sum_f1 += iou_value[4]
ave_IoU = sum_IoU / len(pred_list)
ave_f1 = sum_f1 / len(pred_list)
print(ave_IoU, ave_f1)

tensor(0.6011) tensor(0.7420)


In [34]:
import torch
import torch.nn as nn
import numpy as np

# class IoU(nn.Module):
#     def __init__(self, threshold=0.5):
#         super(IoU, self).__init__()
#         self.threshold = threshold

#     def forward(self, target, input):
#         eps = 1e-10
#         input_ = (input > self.threshold).data.float() # If the value of the pixel is greater than the threshold, then the pixel is 1, otherwise it is 0.
#         target_ = (target > self.threshold).data.float()

#         intersection = torch.clamp(input_ * target_, 0, 1)
#         union = torch.clamp(input_ + target_, 0, 1)

#         if torch.mean(intersection).lt(eps): #less than函数
#             return torch.Tensor([0., 0., 0., 0., 0.])
#         else:
#             acc = torch.mean((input_ == target_).data.float()) # accuracy
#             iou = torch.mean(intersection) / torch.mean(union)
#             recall = torch.mean(intersection) / torch.mean(target_)
#             precision = torch.mean(intersection) / torch.mean(input_)
#             f1 = 2 * recall * precision / (recall + precision) # F1-score

#             return torch.Tensor([acc, recall, precision, iou, f1])

class IoU(nn.Module):
    def __init__(self, threshold=0.5):
        super(IoU, self).__init__()
        self.threshold = threshold

    def forward(self, target, input):
        eps = 1e-10

        # Apply thresholding
        input_ = (input > self.threshold).float()  # Convert to binary: 1 if greater than threshold, else 0
        target_ = (target > self.threshold).float()

        # Calculate intersection and union
        intersection = input_ * target_  # Element-wise multiplication for intersection
        union = input_ + target_  # Element-wise addition for union
        union = torch.clamp(union, 0, 1)  # Ensure union doesn't exceed 1

        # Calculate metrics per image in the batch
        intersection_sum = torch.sum(intersection, dim=(1, 2, 3))  # Sum over batch, height, and width
        union_sum = torch.sum(union, dim=(1, 2, 3))
        target_sum = torch.sum(target_, dim=(1, 2, 3))  # Sum over target for recall calculation
        input_sum = torch.sum(input_, dim=(1, 2, 3))  # Sum over input for precision calculation

        # Avoid division by zero by adding eps
        iou = intersection_sum / (union_sum + eps)
        recall = intersection_sum / (target_sum + eps)
        precision = intersection_sum / (input_sum + eps)

        # Calculate accuracy (pixel-level equality)
        correct_pixels = torch.sum(input_ == target_, dim=(1, 2, 3)).float()
        total_pixels = torch.numel(input_) / input_.size(0)  # Total number of pixels per image
        acc = correct_pixels / total_pixels  # Pixel-level accuracy

        # Calculate F1 score
        f1 = 2 * recall * precision / (recall + precision + eps)

        # Average the results over the batch
        acc_mean = torch.mean(acc)
        iou_mean = torch.mean(iou)
        recall_mean = torch.mean(recall)
        precision_mean = torch.mean(precision)
        f1_mean = torch.mean(f1)

        # Return the results as a tensor
        return torch.Tensor([acc_mean, recall_mean, precision_mean, iou_mean, f1_mean])

iou = IoU()
pred_dir = "/home/fk/python_code/ltqs/weights_bj_baseline/dlink34_sat_gpsdata_png_count_direct_0_dice_bce_loss_1.0__/prediction_npy"
label_dir = "/home/fk/python_code/datasets/dataset_bj_time/test/mask"
pred_list = os.listdir(pred_dir)

# 初始化总和
sum_IoU = 0
sum_f1 = 0
batch_size = 4  # 每个批次包含四张图像

# 获取预测和标签文件列表
pred_list = os.listdir(pred_dir)

# 处理批次
for i in range(0, len(pred_list), batch_size):
    # 获取当前批次的文件
    batch_pred_files = pred_list[i:i+batch_size]
    batch_pred = []
    batch_label = []

    for pred_file in batch_pred_files:
        pred_path = os.path.join(pred_dir, pred_file)
        label_file = pred_file.split(".")[0] + "_mask.png"
        label_path = os.path.join(label_dir, label_file)

        # 加载预测和标签
        pred = torch.Tensor(np.load(pred_path))
        label = torch.Tensor(np.array(Image.open(label_path)))

        # 添加到批次中
        batch_pred.append(pred.unsqueeze(0))  # 增加一个batch维度
        batch_label.append(label.unsqueeze(0))  # 增加一个batch维度

    # 合并为一个批次
    batch_pred = torch.cat(batch_pred, dim=0)  # (batch_size, height, width)
    batch_pred = batch_pred.unsqueeze(1)  # (batch_size, channels, height, width)
    batch_label = torch.cat(batch_label, dim=0)  # (batch_size, height, width)
    batch_label = batch_label.unsqueeze(1)  # (batch_size, channels, height, width)

    iou_value = iou(batch_label, batch_pred)
    sum_IoU += iou_value[3]
    sum_f1 += iou_value[4]
num_batches = len(pred_list) // batch_size
ave_IoU = sum_IoU / num_batches
ave_f1 = sum_f1 / num_batches
print(ave_IoU, ave_f1)

tensor(0.6344) tensor(0.7842)


In [ ]:
import torch
import numpy as np
from PIL import Image
import os

def calculate_batch_iou(label_batch, pred_batch, threshold=0.5):
    """
    计算批量图像的IoU和F1得分。

    :param label_batch: Tensor, 标签图像的批次 (batch_size, height, width)
    :param pred_batch: Tensor, 预测图像的批次 (batch_size, height, width)
    :param threshold: 二值化的阈值
    :return: IoU和F1得分的平均值
    """
    eps = 1e-10
    batch_size = label_batch.size(0)

    # 二值化处理
    pred_batch = (pred_batch > threshold).float()
    label_batch = (label_batch > threshold).float()

    intersection = torch.sum(pred_batch * label_batch, dim=(1, 2))  # 交集
    union = torch.sum(pred_batch, dim=(1, 2)) + torch.sum(label_batch, dim=(1, 2)) - intersection  # 并集

    # 计算 IoU
    iou_batch = intersection / (union + eps)  # 每个图像的IoU

    # 计算 precision, recall 和 F1
    precision_batch = intersection / (torch.sum(pred_batch, dim=(1, 2)) + eps)
    recall_batch = intersection / (torch.sum(label_batch, dim=(1, 2)) + eps)
    f1_batch = 2 * precision_batch * recall_batch / (precision_batch + recall_batch + eps)

    # 返回平均 IoU 和 F1
    return torch.mean(iou_batch), torch.mean(f1_batch)

pred_dir = "/home/fk/python_code/ltqs/weights_bj_baseline/dlink34_sat_gpsdata_png_count_direct_0_dice_bce_loss_1.0__/prediction_npy"
label_dir = "/home/fk/python_code/datasets/dataset_bj_time/test/mask"
pred_list = os.listdir(pred_dir)

# 初始化总和
sum_IoU = 0
sum_f1 = 0
batch_size = 4  # 每个批次包含四张图像

# 处理批次
for i in range(0, len(pred_list), batch_size):
    # 获取当前批次的文件
    batch_pred_files = pred_list[i:i+batch_size]
    batch_pred = []
    batch_label = []

    for pred_file in batch_pred_files:
        pred_path = os.path.join(pred_dir, pred_file)
        label_file = pred_file.split(".")[0] + "_mask.png"
        label_path = os.path.join(label_dir, label_file)

        # 加载预测和标签
        pred = torch.Tensor(np.load(pred_path))
        label = torch.Tensor(np.array(Image.open(label_path)))

        # 添加到批次中
        batch_pred.append(pred.unsqueeze(0))  # 增加一个batch维度
        batch_label.append(label.unsqueeze(0))  # 增加一个batch维度

    # 合并为一个批次
    batch_pred = torch.cat(batch_pred, dim=0)  # (batch_size, height, width)
    batch_label = torch.cat(batch_label, dim=0)  # (batch_size, height, width)

    # 计算当前批次的 IoU 和 F1
    iou_value, f1_value = calculate_batch_iou(batch_label, batch_pred)

    # 累加到总和
    sum_IoU += iou_value.item()
    sum_f1 += f1_value.item()

# 计算平均值
ave_IoU = sum_IoU / (len(pred_list) // batch_size)
ave_f1 = sum_f1 / (len(pred_list) // batch_size)

print("Average IoU:", ave_IoU)
print("Average F1-score:", ave_f1)


Average IoU: 0.6343679831308477
Average F1-score: 0.7842257689027226


In [ ]:
import torch

def calculate_iou(preds, labels, threshold=0.5):
    """
    计算IoU指标
    preds: 预测值，Tensor类型，大小为(B, H, W)，其中B是batch size，H是高度，W是宽度
    labels: 真实标签，Tensor类型，大小为(B, H, W)，其中B是batch size，H是高度，W是宽度
    threshold: 预测值的阈值，默认为0.5
    返回 IoU 值
    """

    # 将预测结果应用阈值（假设预测结果是概率值）
    preds = preds > threshold  # 二值化
    labels = labels > threshold  # 二值化

    # 扁平化处理
    preds = preds.view(-1)
    labels = labels.view(-1)

    # 计算交集与并集（使用逻辑运算）
    intersection = torch.logical_and(preds, labels).sum().float()  # 交集
    union = torch.logical_or(preds, labels).sum().float()        # 并集

    # 计算IoU，避免除以0
    iou = intersection / (union + 1e-6)

    return iou

pred_dir = "/home/fk/python_code/ltqs/weights_bj_baseline/dlink34_sat_gpsdata_png_count_direct_0_dice_bce_loss_1.0__/prediction_npy"
label_dir = "/home/fk/python_code/datasets/dataset_bj_time/test/mask"
pred_list = os.listdir(pred_dir)
sum_IoU = 0
for pred_file in pred_list:
    pred_path = os.path.join(pred_dir, pred_file)
    label_file = pred_file.split(".")[0] + "_mask.png"
    label_path = os.path.join(label_dir, label_file)
    pred = torch.Tensor(np.load(pred_path))
    label = torch.Tensor(np.array(Image.open(label_path)))

    iou_value = calculate_iou(pred, label)
    sum_IoU += iou_value.item()
ave_IoU = sum_IoU / len(pred_list)
print(ave_IoU)

0.6010629245213099
